In [ ]:
# Ipython 
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
# Sympy
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel

# Double pendulum, two point masses, angles wrt x axis

```
 Coordinate system:
 O --------------> y 
 |\
 |~\
 |t1\
 |   o (point mass 1)
 |   |~\
 |   |t2\
 |   |   \
 |   |    o (point mass 2)
 x   
 ```

In [ ]:
# --- OPTION 1 - The angles are about the reference frame (x axis)
m, l = sp.symbols('m, l')
m1, l1, m2, l2 = sp.symbols('m1, l1, m2, l2')
# HACK, comment if undesired
#m1, m2, l1, l2 = m, m, l ,l
gravity = sp.symbols('g')
time = dynamicsymbols._t
theta1, theta2 = dynamicsymbols('theta_1, theta_2')
theta1d, theta2d = dynamicsymbols('theta_1d, theta_2d')
# --- Bodies
ref = YAMSInertialBody('E') 
pend1 = YAMSRigidBody('P1', mass=m1, J_G=(0,0,0), rho_G=(l1,0,0)) 
pend2 = YAMSRigidBody('P2', mass=m2, J_G=(0,0,0), rho_G=(l2,0,0)) 
bodies=[pend1,pend2] 
# --- Connections
ref.connectTo(pend1, type='Joint', rel_pos=(0, 0, 0), rot_type='Body', rot_amounts=(0,0,theta1), rot_order='XYZ')
#pend1.connectTo(pend2, type='Joint', rel_pos=(l, 0, 0), rot_type='Body', rot_amounts=(0,0,theta2), rot_order='XYZ')
pend1.connectTo(pend2, type='Joint', rel_pos=(l1, 0, 0), rot_type='Body', rot_amounts=(0,0,theta2), rot_order='XYZ', ref_frame=ref.frame)
# --- DOFs and kinetic equations
coordinates = [theta1,theta2]
speeds      = [theta1d,theta2d]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, bdy.mass * gravity * ref.frame.x))] # NOTE: gravity along x
# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
EOM=model.to_EOM(simplify=True)
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F
print('Center of mass of second mass:')
pend2.masscenter_pos_global.express(ref.frame)


In [ ]:
# --- Small angle approximation
EOMsm = EOM.smallAngleApprox(coordinates)
EOMsm.mass_forcing_form()
EOMsm.M
EOMsm.F
# --- Linearization of full EOM (NOTE: linearization about non zero velocity could fail without special care)
#M0, C0, K0, B0 = EOM.linearize(op_point=[(sp.diff(theta1,time),symbols('t1_d')),(sp.diff(theta2,time),symbols('t2_d')),(theta1,0),(theta2,0)], noAcc=True, noVel=False, extraSubs=None, simplify=True)
M0, C0, K0, B0 = EOM.linearize(op_point=[(sp.diff(theta1,time),0),(sp.diff(theta2,time),0),(theta1,0),(theta2,0)], noAcc=True, noVel=False, simplify=True)
M0
K0
C0
# --- Linearization of small angle EOM (NOTE: linearization about non zero velocity could fail without special care)
#M0, C0, K0, B0 = EOMsm.linearize(op_point=None, noAcc=True, noVel=False, extraSubs=None, simplify=True)
#M0, C0, K0, B0 = EOMsm.linearize(op_point=[(sp.diff(theta1,time),symbols('t1_d')),(theta1,0),(theta2,0)], noAcc=True, noVel=False, extraSubs=None)
M0, C0, K0, B0 = EOMsm.linearize(noAcc=True, noVel=True, simplify=True)
M0
K0
C0



In [ ]:
#from welib.yams.yams_sympy_tools import cleantex, cleanPy
#print(sp.latex(EOM.M))
#print(cleantex(EOM.M))
#print(cleantex(EOM.F))

In [ ]:
# --- Layer 1, purely mechanics
from sympy import symbols
from sympy.physics.mechanics import *
q1, q2 = dynamicsymbols('q1 q2')
q1d, q2d = dynamicsymbols('q1 q2', 1)
u1, u2 = dynamicsymbols('u1 u2')
u1d, u2d = dynamicsymbols('u1 u2', 1)
l, m, g = symbols('l m g')
N = ReferenceFrame('N')
A = N.orientnew('A', 'Axis', [q1, N.z])
B = N.orientnew('B', 'Axis', [q2, N.z])
A.set_ang_vel(N, u1 * N.z)
B.set_ang_vel(N, u2 * N.z)
O = Point('O')
P = O.locatenew('P', l * A.x)
R = P.locatenew('R', l * B.x)
O.set_vel(N, 0)
P.v2pt_theory(O, N, A)
R.v2pt_theory(P, N, B)
ParP = Particle('ParP', P, m)
ParR = Particle('ParR', R, m)
kd = [q1d - u1, q2d - u2]
FL = [(P, m * g * N.x), (R, m * g * N.x)]
BL = [ParP, ParR]
KM = KanesMethod(N, q_ind=[q1, q2], u_ind=[u1, u2], kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BL, FL)
kdd = KM.kindiffdict()
mm = KM.mass_matrix_full
fo = KM.forcing_full
kdSubs=[(q1d, u1), (q2d, u2)]
mm=mm.subs(kdSubs)
fo=fo.subs(kdSubs)
mm.simplify()
fo.simplify()
mm
fo

# Double pendulum, second angle is relative to first one

In [ ]:
# --- OPTION 2 - The second angle is relative to the first one
m, l = sp.symbols('m, l')
gravity = sp.symbols('g')
time = dynamicsymbols._t
theta1, theta2 = dynamicsymbols('theta_1, theta_2')
theta1d, theta2d = dynamicsymbols('theta_1d, theta_2d')
# --- Bodies
ref = YAMSInertialBody('E') 
pend1 = YAMSRigidBody('P1', mass=m, J_G=(0,0,0), rho_G=(l,0,0)) 
pend2 = YAMSRigidBody('P2', mass=m, J_G=(0,0,0), rho_G=(l,0,0)) 
bodies=[pend1,pend2] 
# --- Connections
ref.connectTo(pend1, type='Joint', rel_pos=(0, 0, 0), rot_type='Body', rot_amounts=(0,0,theta1), rot_order='XYZ')
pend1.connectTo(pend2, type='Joint', rel_pos=(l, 0, 0), rot_type='Body', rot_amounts=(0,0,theta2), rot_order='XYZ')
#pend1.connectTo(pend2, type='Joint', rel_pos=(l, 0, 0), rot_type='Body', rot_amounts=(0,0,theta2), rot_order='XYZ', ref_frame=ref.frame)
# --- DOFs and kinetic equations
coordinates = [theta1,theta2]
speeds      = [theta1d,theta2d]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, bdy.mass * gravity * ref.frame.x))] # NOTE: gravity along x
# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
EOM=model.to_EOM(simplify=True)
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F
print('Center of mass of second mass:')
pend2.masscenter_pos_global.express(ref.frame)
#M0, C0, K0, B0 = EOM = EOM.linearize(op_point=None, noAcc=True, noVel=False, extraSubs=None)